## Temat projektu:


Określanie pozycji na której powinien grać piłkarz na podstawie jego cech  z wykorzystaniem poznanych narzędzi oraz w środowisku MS Azure. 


## Autorzy: 

Stanisław Frączkowski 

Jan Zubalewicz 

Kamil Bilski

## Cel projektu:

 
Celem projektu jest określenie pozycji na której powinien grać dany piłkarz na podstawie jego statystyk. Dane do projektu zostały wzięte z gry piłkarskiej FIFA 19.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/kafagy/fifa-FUT-Data/master/FIFA19.csv')

# liczba obiektów
liczba_obiektow = df.shape[0]
print("liczba obiektów:", liczba_obiektow)

#liczba atrybutow
liczba_atrybutow = df.shape[1]
print("liczba atrybutow:", liczba_atrybutow)

# liczba klas
print("liczba klas:", len(df['POSITION'].unique()))

# liczba obiektów w każdej klasie
print("Obiektów w klasie: ")
for i in list(df['POSITION'].unique()):
    print(i , ' : ', len(df[df['POSITION'] == i]))

# liczba danych brakujących
print("brakujące:", df.isnull().sum().sum())

# usunięcie danych brakujących
df = df.dropna()


liczba obiektów: 13633
liczba atrybutow: 13
liczba klas: 14
Obiektów w klasie: 
CAM  :  872
ST  :  2167
CF  :  107
LW  :  313
CB  :  2650
CM  :  1820
RW  :  295
RB  :  1117
LM  :  911
LB  :  1135
CDM  :  1224
RM  :  933
RWB  :  45
LWB  :  44
brakujące: 0


## Analiza eksploracyjna:

Zbiór składa się z 13 atrybutów, oraz z 13633 obiektów.  Każdy obiekt zawiera imię, klub, ligę oraz pozycję piłkarza. Posiada także klasyfikację ze względu na poziom piłkarza(Gold, silver oraz bronze).  Zbiór zawiera także podstawowe statystyki piłkarza(szybkość, strzały, podania, drybling, obrona oraz fizyczność). Statystyki są podane w skali 0-99. Na jej podstawie została także wyliczona ocena ogólna piłkarza(także w skali 0-99). Zbiór posiada także atrybut określający datę i godzinę wstawienia obiektu do zbioru. Najbardziej interesującym atrybutem w kontekście klasyfikacji będzie dla nas pozycja danego piłkarza. Pozycje zostały podzielone na 14 klas(określających pozycję piłkarza na boisku).  

Opis atrybutów zbioru danych:  

NAME – Imię i nazwisko zawodnika 

CLUB – aktualny klub zawodnika 

LEAGUE – liga w której gra aktualny klub zawodnika 

POSITION – pozycja, na jakiej gra zawodnik (pozycje są opisane niżej) 

TIER – poziom zawodnika (złoto/srebro/brąz) 

RATING – ogólna ocena zawodnika, wartość liczbowa 0-99 

PACE – szybkość, wartość liczbowa 0-99 

SHOOTING – strzały, wartość liczbowa 0-99 

PASSING – podania, wartość liczbowa 0-99 

DRIBBLING – drybling, wartość liczbowa 0-99 

DEFENDING – obrona/defensywa, wartość liczbowa 0-99 

PHYSICAL – sprawność fizyczna, wartość liczbowa 0-99 

LOADDATE – data załadowania do zbioru danych, nieużywana przez nas 

 

Statystyki klas: 

CAM  :  872, ST  :  2167, CF  :  107, LW  :  313, CB  :  2650, CM  :  1820, RW  :  295, RB  :  1117, 

LM  :  911, LB  :  1135, CDM  :  1224, RM  :  933, RWB  :  45, LWB  :  44 

Opis skrótów pozycji:  

ST - napastnik  

CB - obrońca centralny  

LB - obrońca lewy  

RB - obrońca prawy  

CDM - defensywny pomocnik  

CM - środkowy pomocnik  

CAM - ofensywny pomocnik  

LM - lewy pomocnik  

RM - prawy pomocnik  

LW - lewy skrzydłowy  

RW - prawy skrzydłowy 

 

Zbiór nie zawiera brakujących danych. 

W celu polepszenia jakości nauczania nadzorowanego musieliśmy odfiltrować ze zbioru danych przypadki skrajne, gdzie jedna ze statystyk znacząco odstaje i przypadki zupełnie przeciętne, w których wyniki w konkretnych polach nie różnią się wystarczająco, by można było znaleźć powiązanie z konkretną pozycją. W przypadkach skrajnych usunęliśmy napastników, którzy mają lepszą obronę niż atak, graczy defensywnych z lepszym atakiem niż obroną. Poza tym, w celu zróżnicowania bazy do nauczania, podzieliliśmy zbiór danych na 3 rozłączne zbiory : 

Icons – Gracze dodani jako ciekawostka, najlepsi gracze wszech czasów (np. Pele, Diego Maradona itd.) 

Gold – Gracze z kategorią „Złoto” w rankingu (najlepsi aktualnie grający) 

Mid – Gracze z kategoriami „Srebro” i „Brąz” w rankingu (pozostali) 

In [3]:
# usuwanie napastników mających lepszą obronę niż atak
df = df[(df.POSITION != 'ST') | (df.SHOOTING >= df.DEFENDING)]

# obliczenie odchylenia standardowego dla kolumn ze statystykami
stats_cols = ['PACE', 'SHOOTING', 'PASSING', 'DRIBBLING', 'DEFENDING', 'PHYSICAL']

#usunięcie graczy defensywnych z lepszym atakiem niż defensywą
df = df[((df.POSITION != 'CB') & (df.POSITION != 'LB') & (df.POSITION != 'RB') & (df.POSITION != 'CB') & (df.POSITION != 'GK')) | (df.SHOOTING <= df.DEFENDING)]



## Rozwiązanie problemu: 

Problem został rozwiązany przy pomocy klasyfikatora k-Najbliższych Sąsiadów. Tworzymy na początku zbiór uczący(używamy 70% danych z projektu) oraz tworzymy model KNN. Został on wybrany z powodu takiego, że dla tej metody klasyfikacja będzie w tym przypadku najlepsza. Metoda najbliższego sąsiada tutaj się nadawałaby, gdyż powodowałaby zbyt dużo błędów. Naiwny klasyfikator Bayesa nie daje możliwości ustalenia ilu sąsiadów mamy wziąć pod uwagę przy klasyfikacji. Zależało nam na tym gdyż dzielimy nasze dane na 3 podgrupy zawierające różną liczbę danych, więc wybór ilości sąsiadów do klasyfikacji daja dużą elastyczność. Metodą prób i błędów dla kart złotych wybraliśmy klasyfikację na podstawie 20 sąsiadów, a dla srebrnych i brązowych 25(zostało to zrobione w ten sposób gdyż kart srebrnych i brązowych jest więcej). Niestety, dla zbioru ikon okazało się, że niektórych danych w klasach jest zbyt mało danych przez co nie byliśmy w stanie przeprowadzić dla tych kart klasyfikacji. Następnie dany model zostaje stestowany.  

In [4]:
# Podział na grupy według rankingu
df_icons = df[df.LEAGUE == 'Icons']

df_gold = df[(df.TIER == 'Gold') & (df.LEAGUE != 'Icons')]

df_mid = df[((df.TIER == 'Silver') | (df.TIER == 'Bronze')) & (df.LEAGUE != 'Icons')]


## Wyniki:

Po podzieleniu danych na 3 podgrupy, uczymy model, następnie wykonujemy klasyfikację. 

In [5]:
# Uczenie dla kart złotych
X  = df_gold.iloc[:,6:12].values
y  = df_gold['POSITION'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, stratify=y)

In [6]:
# stworzenie instancji modelu i wpasowanie tego modelu do danych trenujących
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train, y_train)

#Przykładowe wywołanie
ronaldo = np.array([[90,90,90,90,50,70]])
van_dijk = np.array([[60,40,50,60,90,90]])
print(knn.predict(ronaldo))
print(knn.predict(van_dijk))

['LM']
['CB']


In [7]:
from sklearn.preprocessing import StandardScaler

#Standaryzacja
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

Szukamy najlepszego modelu klasyfikującego. Iteracyjnie szukamy optymalnego modelu, przechodząc przez 40 sąsiadów 

In [8]:
# przeprowazdenie piewszego testu dokładności
n = 40
best_acc = 0

for i in range(1, n):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_nei = i
        
        
print("Best acc:", best_acc)
print("Best neig:", best_nei)

Best acc: 0.21962616822429906
Best neig: 32


Wylicza dokładność naszego modelu dla wszystkich użytych danych i pozycji.

In [9]:
# przeprowazdenie drugiego testu dokładności 
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.16822429906542055


Wylicza dokładność naszego modelu dla poszczególnych pozcyji oraz wszystkich pozycji. 

In [10]:
# sprawdzenie dokładnoości testu dla konkretnych pozycji
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         CAM       0.00      0.00      0.00        19
          CB       0.17      0.89      0.29        18
         CDM       0.14      0.07      0.09        30
          CF       0.00      0.00      0.00         2
          CM       0.22      0.25      0.24        52
          LB       0.00      0.00      0.00        13
          LM       0.00      0.00      0.00        18
          LW       0.00      0.00      0.00         9
         LWB       0.00      0.00      0.00         1
          RB       0.00      0.00      0.00         8
          RM       0.00      0.00      0.00        15
          RW       0.00      0.00      0.00         7
         RWB       0.00      0.00      0.00         1
          ST       0.17      0.24      0.20        21

    accuracy                           0.17       214
   macro avg       0.05      0.10      0.06       214
weighted avg       0.11      0.17      0.11       214



c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [11]:
#Uczenie dla kart srebrnych i brązowych
X  = df_mid.iloc[:,6:12].values
y  = df_mid['POSITION'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, stratify=y)

In [12]:
# stworzenie instancji modelu i wpasowanie tego modelu do danych trenujących
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)

#Przykładowe wywołanie
ronaldo = np.array([[90,90,90,70,30,60]])
van_dijk = np.array([[30,30,40,50,90,90]])
print(knn.predict(ronaldo))
print(knn.predict(van_dijk))

['ST']
['CB']


In [13]:
from sklearn.preprocessing import StandardScaler

#Standaryzacja
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

Szukamy najlepszego modelu klasyfikującego. Interacyjnie szukamy optymalnego modelu, przechodząc przez 40 sąsiadów 


In [14]:
# przeprowazdenie piewszego testu dokładności
n = 40
best_acc = 0

for i in range(1, n):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_nei = i
        
        
print("Best acc:", best_acc)
print("Best neig:", best_nei)

Best acc: 0.2117465224111283
Best neig: 38


Wylicza dokładność naszego modelu dla wszystkich użytych danych i pozycji.

In [15]:
# przeprowazdenie drugiego testu dokładności 
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.20865533230293662


Wylicza dokładność naszego modelu dla poszczególnych pozcyji oraz wszystkich pozycji. 

In [16]:
# sprawdzenie dokładności testu dla konkretnych pozycji
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         CAM       0.10      0.01      0.02       100
          CB       0.22      0.70      0.34       186
         CDM       0.19      0.11      0.14       141
          CF       0.00      0.00      0.00        12
          CM       0.19      0.26      0.22       224
          LB       0.00      0.00      0.00        88
          LM       0.09      0.02      0.03        99
          LW       0.00      0.00      0.00        46
         LWB       0.00      0.00      0.00         9
          RB       0.12      0.03      0.04        80
          RM       0.00      0.00      0.00       130
          RW       0.00      0.00      0.00        36
         RWB       0.00      0.00      0.00         5
          ST       0.28      0.43      0.34       138

    accuracy                           0.21      1294
   macro avg       0.08      0.11      0.08      1294
weighted avg       0.14      0.21      0.14      1294



c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [17]:
#Uczenie dla kart icons
X  = df_icons.iloc[:,6:12].values
y  = df_icons['POSITION'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7)

In [18]:
# stworzenie instancji modelu i wpasowanie tego modelu do danych trenujących
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)

#Przykładowe wywołanie
ronaldo = np.array([[90,90,90,70,30,60]])
van_dijk = np.array([[30,30,40,50,90,90]])
print(knn.predict(ronaldo))
print(knn.predict(van_dijk))

['ST']
['CAM']


In [19]:
from sklearn.preprocessing import StandardScaler

#Standaryzacja
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Szukamy najlepszego modelu klasyfikującego. Interacyjnie szukamy optymalnego modelu, przechodząc przez 40 sąsiadów 


In [20]:
# przeprowazdenie piewszego testu dokładności
n = 40
best_acc = 0

for i in range(1, n):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_nei = i
        
        
print("Best acc:", best_acc)
print("Best neig:", best_nei)

Best acc: 0.34285714285714286
Best neig: 14


Wylicza dokładność naszego modelu dla wszystkich użytych danych i pozycji.

In [21]:
# przeprowazdenie drugiego testu dokładności 
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = knn.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.22857142857142856


Wylicza dokładność naszego modelu dla poszczególnych pozcyji oraz wszystkich pozycji.

In [22]:
# sprawdzenie dokładności testu dla konkretnych pozycji
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         CAM       0.00      0.00      0.00         4
          CB       0.00      0.00      0.00         3
         CDM       0.00      0.00      0.00         2
          CF       0.00      0.00      0.00         6
          CM       0.50      0.17      0.25         6
          LM       0.00      0.00      0.00         1
          LW       0.00      0.00      0.00         3
          RW       0.00      0.00      0.00         1
          ST       0.37      0.78      0.50         9

    accuracy                           0.23        35
   macro avg       0.10      0.10      0.08        35
weighted avg       0.18      0.23      0.17        35



c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\kamil\pycharmprojects\pythonproject2\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

## Wnioski:

Podsumowując skuteczność tego modelu nie jest zbyt wysoka. Problemem tutaj okazały się dane, myślę że wykonując projekt innym sposobem klasyfikacji rezultat byłby podobny. Pomimo zastosowania różnych technik filtracji oraz dzielenia dane na podgrupy, skuteczność nie zwiększyła się znacząco. Dla lepszego zrozumienia problematyki danych, podam przykład. Napastników cechuje wyższa niż reszta atrybutów ocena strzałów, oraz zazwyczaj dużą szybkość biegu. Jednakże, zdarzają się tacy napastnicy którzy tej szybkości biegu mają relatywnie mało. Takie dane psują mocno klasyfikację. Problemem jest nawet wyfiltrować je, ponieważ warto też zauważyć, że piłkarze są różni i grając na tej samej pozycji mogą mieć różne zadania na boisku oraz mogą być innej charakterystyki, co wpływa także na ich odzwierciedlenie statystyk w FIFIE. 